In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from IPython.display import Audio
sound_file = './beep-01a.wav'

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test_v2.csv')

### Because the purpose of this movement is to interpolate risk factor, we are justified to bring in the testing data to train this model

In [3]:
# Get a sense of what columns the dataset contains
print np.unique(train_data.columns)

['A' 'B' 'C' 'C_previous' 'D' 'E' 'F' 'G' 'age_oldest' 'age_youngest'
 'car_age' 'car_value' 'cost' 'customer_ID' 'day' 'duration_previous'
 'group_size' 'homeowner' 'location' 'married_couple' 'record_type'
 'risk_factor' 'shopping_pt' 'state' 'time']


In [8]:
merged_data = train_data.append(test_data, ignore_index=True)
merged_data['average_age'] = (merged_data.age_oldest - merged_data.age_youngest) / 2

In [18]:
merged_data_risk_notnull = merged_data[merged_data.risk_factor.notnull()]
merged_data_risk_predictors = merged_data_risk_notnull[['state', 'location', 
                                         'homeowner', 'car_age',
                                         'car_value', 'average_age', 
                                         'married_couple', 'duration_previous']]
merged_data_risk_label = merged_data_risk_notnull.risk_factor

In [31]:
# Find out what columns have empty values
def any_null(series):
    return any(series.isnull())
merged_data_risk_predictors.apply(any_null)

state                False
location             False
homeowner            False
car_age              False
car_value            False
average_age          False
married_couple       False
duration_previous    False
dtype: bool

In [20]:
# Interpolate customer location by using the mode of the locations in the state
mode = lambda x: int(scipy.stats.mode(x)[0][0])
state_location_mode = merged_data_risk_predictors.groupby('state')['location'].apply(mode)
Audio(url=sound_file, autoplay=True)

In [25]:
# Fill empty location cells with the location mode of the corresponding states
get_state_location = lambda state: state_location_mode[state]
merged_data_empty_location = merged_data_risk_predictors[merged_data_risk_predictors.location.isnull()]
#merged_data_risk_predictors.state.apply(get_state_location)
merged_data_empty_location.location = merged_data_empty_location.state.apply(get_state_location)
merged_data_risk_predictors[merged_data_risk_predictors.location.isnull()] = merged_data_empty_location

D:\Anaconda2\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [30]:
# Assign 'none' for all blank car values, because it can be assumed that those customers with that column blank have no cars. 
# They belong to a separate class
merged_data_risk_predictors.car_value[merged_data_risk_predictors.car_value.isnull()] = 'n'

# Assign 0 for all blank duration_previous, because it can be assumed that these customers are new
merged_data_risk_predictors.duration_previous[merged_data_risk_predictors.duration_previous.isnull()] = 0

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\Anaconda2\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
D:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: Set

In [ ]:
# Check that the dimension for 

In [ ]:
# Check the length of the dataset
